In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'<GROQ_API_KEY>'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(api_key=groq_api_key, model="Gemma2-9b-It")
model

ModuleNotFoundError: No module named 'langchain_groq'

In [42]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello! My name is Rohit.")])

AIMessage(content="Hello Rohit! It's nice to meet you. 👋 \n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.001267688, 'queue_time': 0.2531614, 'total_time': 0.046722233}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--20913c7f-2def-4439-a8a2-57b3682fec65-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41})

In [43]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hello! My name is Rohit."), AIMessage(content="Hello Rohit! How can I assist you today?"),
HumanMessage(content="Can you tell me a joke?"),
])

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! \n\nLet me know if you'd like to hear another one. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 43, 'total_tokens': 78, 'completion_time': 0.063636364, 'prompt_time': 0.001629637, 'queue_time': 0.249762341, 'total_time': 0.065266001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c567c886-1c51-450d-9fbd-5f64b7d5f310-0', usage_metadata={'input_tokens': 43, 'output_tokens': 35, 'total_tokens': 78})

In [6]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [44]:
## chat message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model, get_session_history=get_session_history
)

In [45]:
consfig = {
   "configurable":{"session_id":"chat1"},
}

In [46]:
response=with_message_history.invoke([HumanMessage(content="Hello! My name is Rohit.")], config=consfig)

In [47]:
response.content

"Hello Rohit, it's nice to meet you! 👋 \n\nWhat can I do for you today?\n"

In [28]:
with_message_history.invoke([HumanMessage(content="Hello! What is my Name?")], config=consfig)

AIMessage(content='Your name is Rohit.  \n\nYou told me earlier! 😊  \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 57, 'total_tokens': 75, 'completion_time': 0.032727273, 'prompt_time': 0.002188109, 'queue_time': 0.250650401, 'total_time': 0.034915382}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2876fa0c-9900-4060-a1f5-9783f749957c-0', usage_metadata={'input_tokens': 57, 'output_tokens': 18, 'total_tokens': 75})

In [48]:
consfig1 = {
   "configurable":{"session_id":"chat2"},
}
response=with_message_history.invoke([HumanMessage(content="What is my Name?")], config=consfig1)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [33]:
response=with_message_history.invoke([HumanMessage(content="My Name is John?")], config=consfig1)
response.content

"Hello John! It's nice to meet you.  \n\nIs there anything I can do for you today? \n"

In [49]:
with_message_history.invoke([HumanMessage(content="Hello! What is my Name?")], config=consfig1)

AIMessage(content="Since I don't have access to past conversations, I don't know your name.  \n\nWould you like to tell me? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 71, 'total_tokens': 105, 'completion_time': 0.061818182, 'prompt_time': 0.002295995, 'queue_time': 0.250392114, 'total_time': 0.064114177}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--337b7a01-8d76-4962-9eec-bacb92436a45-0', usage_metadata={'input_tokens': 71, 'output_tokens': 34, 'total_tokens': 105})

In [51]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
prompt=ChatPromptTemplate.from_messages([
("system","You are a helpful assistant that remembers the user's name."),
MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [52]:
chain.invoke({"messages":[HumanMessage(content="Hello! My name is Rohit.")]}).content

"Hello Rohit! It's nice to meet you. How can I help you today? 😊  \n\n"

In [53]:
with_message_history=RunnableWithMessageHistory(
    chain, get_session_history=get_session_history
)

In [55]:
config = {
   "configurable":{"session_id":"chat3"},
}

response=with_message_history.invoke([HumanMessage(content="What is my Name?")], config=config)
response.content

"Since we've just met, I don't know your name!  \n\nTell me, what would you like me to call you? 😄  \n"